In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import tensorflow as tf
# prepare data for machine learning model
# https://machinelearningmastery.com/random-forest-for-time-series-forecasting/
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)


# Given univariate time series (equal datasteps)
raw_seq = pd.Series([91.57, 91.56, 91.40, 91.39, 91.20, 90.90, 90.99,
                     91.17, 90.98, 91.11, 91.20, 90.92, 90.73, 90.79, 90.86, 90.83,
                     90.80, 90.21, 90.10, 90.10, 89.66, 89.81, 89.40, 89.34, 89.16,
                     88.71, 89.28, 89.36, 89.64, 89.53, 89.32, 89.14, 89.41, 89.53,
                     89.16, 88.98, 88.50, 88.63, 88.62, 88.76, 89.07, 88.47, 88.41,
                     88.57, 88.23, 87.93, 87.88, 88.18, 88.04, 88.18, 88.78, 89.29,
                     89.14, 89.14, 89.42, 89.26, 89.37, 89.51, 89.66, 89.39, 89.02,
                     89.05, 88.97, 88.57, 88.44, 88.52, 87.92, 87.71, 87.52, 87.37,
                     87.27, 87.07, 86.83, 86.88, 87.48, 87.30, 87.87, 87.85, 87.83,
                     87.40, 86.89, 87.38, 87.48, 87.21, 87.02, 87.10, 87.02, 87.07,
                     86.74, 86.35, 86.32, 86.77, 86.77, 86.49, 86.02, 85.52, 85.02,
                     86.02, 85.52, 85.02, 84.42, 85.29])
"""
------------------------------------------------------------
Analysis of Time Series
------------------------------------------------------------
"""
# visualization
raw_seq.plot()
plt.xlabel("Time")
plt.title("Raw sequence")
plt.show()

# Detrending
from scipy import signal
detrended = signal.detrend(raw_seq.values)
plt.plot(detrended)
plt.title('Raw sequence detrended by subtracting the least squares fit')
plt.xlabel("Time")
plt.show()

# todo: modify the code to deseasonalize with the assumption of additive model. Compare the results.
# Seasonal decomposition for multiplicative model
from statsmodels.tsa.seasonal import seasonal_decompose
result_mul = seasonal_decompose(raw_seq, model='multiplicative', period=12)
# Deseasonalize
deseasonalized = raw_seq.values / result_mul.seasonal
# Plot
plt.plot(deseasonalized)
plt.title('Deseasonalized for multiplicative model')
plt.xlabel("Time")
plt.show()

## Test for seasonality
from pandas.plotting import autocorrelation_plot
# Draw Plot (for seasonal data, multiple peaks should appear)
autocorrelation_plot(raw_seq.values.tolist())
plt.show()

# check the histogram of values
plt.hist(raw_seq)
plt.show()

# todo: modify the function to check if time series is stationary by comparing custom number of intervals
# check if the time series is stationary (has the same characteristics in both intervals)
def timeSeriesSplitMeanVar(ts):
    split = len(ts) // 2
    X1, X2 = ts[0:split], ts[split:]
    mean1, mean2 = X1.mean(), X2.mean()
    var1, var2 = X1.var(), X2.var()
    print('mean1=%f, mean2=%f' % (mean1, mean2))
    print('variance1=%f, variance2=%f' % (var1, var2))
timeSeriesSplitMeanVar(raw_seq)

# todo: check if the raw_seq, deseasonalized and detrended time series are stationary in two and more intervals

"""
------------------------------------------------------------
ARIMA
------------------------------------------------------------
"""
from statsmodels.tsa.arima.model import ARIMA
model = ARIMA(raw_seq, order=(1, 0, 1))
model_fit = model.fit()

forecast = model_fit.get_forecast(steps=10)
# Split the data into train and test
train_size = int(len(raw_seq) * 0.8)
train, test = raw_seq[0:train_size], raw_seq[train_size:len(raw_seq)]

# todo: use at least 6 combinations of ARIMA parameters and compare the obtained results
# Fit the ARIMA model on the training dataset
model_train = ARIMA(train, order=(5, 2, 1)) # (p, d, q)
model_train_fit = model_train.fit()

# Forecast on the test dataset
test_forecast = model_train_fit.get_forecast(steps=len(test))
test_forecast_series = pd.Series(test_forecast.predicted_mean, index=test.index)

# Calculate the mean squared error
mse = mean_squared_error(test, test_forecast_series)
plt.plot(train, label='Training Data')
plt.plot(test, label='Actual Data', color='orange')
plt.plot(test_forecast_series, label='Forecasted Data', color='green')
plt.title('ARIMA Model Evaluation')
plt.xlabel('Date')
plt.ylabel('value')
plt.legend()
plt.show()

"""
------------------------------------------------------------
Random forest
------------------------------------------------------------
"""


# choose a number of time steps
n_steps = 3

# validate the model for one step forward (valid_no times)
# todo: compare the validation error with different number of estimators
# todo: compare the validation error with different number of previous values used in the model
valid_no = 10
raw_seq.plot(color = "blue")
y_valid = list()
for i in range(valid_no):
    # split into samples
    X , y = split_sequence(raw_seq[:(-valid_no+i)], n_steps)
    model = RandomForestRegressor(n_estimators=50)
    model.fit(X, y)
    x_input = np.array(raw_seq[(-valid_no+i-n_steps):(-valid_no+i)])
    x_input = x_input.reshape((1, n_steps))
    yhat = model.predict(x_input)
    y_valid.append(yhat)

plt.plot(np.arange((len(raw_seq)-valid_no),(len(raw_seq)), 1), y_valid,'mo',label = "one-step validation")
print(mean_squared_error(raw_seq[(-valid_no):], y_valid))
plt.show()

x_input = np.array(raw_seq[(-n_steps):])

xTest = pd.DataFrame([85.32])
x_input = x_input.reshape((1, n_steps))
yhat = model.predict(x_input)
print('predicted=%f, actual=%f' % (yhat, xTest[0][0]))
raw_seq.plot(color = 'b', label = "train")
plt.plot(len(raw_seq), yhat, 'ro', label = "predicted")
plt.plot(len(raw_seq), xTest[0][0], 'go', label = "actual")
plt.legend()
plt.xlabel("time")
plt.ylabel("value")
plt.show()
"""
------------------------------------------------------------
MLP
------------------------------------------------------------
"""
# todo: compare the test error with different number of layers and neurons
# todo: compare the test error with different number of previous values used in the model
import matplotlib.pyplot as plt
MLPmodel = tf.keras.Sequential()
MLPmodel.add(tf.keras.layers.Dense(100, activation='relu', input_dim=n_steps))
MLPmodel.add(tf.keras.layers.Dense(1))
MLPmodel.compile(optimizer='adam', loss='mse')
model_history = MLPmodel.fit(X, y, epochs=20, verbose=1)
plt.plot(model_history.history['loss'], label='train')
plt.legend()
plt.xlabel("epochs")
plt.ylabel("loss function")
plt.show()
x_input = np.array(raw_seq[(-n_steps):])

xTest = pd.DataFrame([85.32])
x_input = x_input.reshape((1, n_steps))
yhat = MLPmodel.predict(x_input)
print('predicted=%f, actual=%f' % (yhat[0][0], xTest[0][0]))
raw_seq.plot(color = 'b', label = "train")
plt.plot(len(raw_seq), yhat, 'ro', label = "predicted")
plt.plot(len(raw_seq), xTest[0][0], 'go', label = "actual")
plt.legend()
plt.xlabel("time")
plt.ylabel("value")
plt.show()






ModuleNotFoundError: No module named 'sklearn'